In [15]:
import pandas as pd

path = "../data/valeurs_foncieres_2024.csv"

# Infos

In [16]:
df = pd.read_csv(path)
display(df.head(3)) 
df.info()

C:\Users\jadep\AppData\Local\Temp\ipykernel_15900\4026636684.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2024-802852,2024-01-05,1,Vente,350000.0,28.0,B,RUE JEAN CLAUDE BARTET,0160,69410.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,4.794854,45.791146
1,2024-802852,2024-01-05,1,Vente,350000.0,28.0,B,RUE JEAN CLAUDE BARTET,0160,69410.0,...,Appartement,69.0,3.0,NaN,NaN,NaN,NaN,NaN,4.794854,45.791146
2,2024-802852,2024-01-05,1,Vente,350000.0,28.0,B,RUE JEAN CLAUDE BARTET,0160,69410.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,4.794854,45.791146


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74593 entries, 0 to 74592
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_mutation                   74593 non-null  object 
 1   date_mutation                 74593 non-null  object 
 2   numero_disposition            74593 non-null  int64  
 3   nature_mutation               74593 non-null  object 
 4   valeur_fonciere               74316 non-null  float64
 5   adresse_numero                60516 non-null  float64
 6   adresse_suffixe               3976 non-null   object 
 7   adresse_nom_voie              73785 non-null  object 
 8   adresse_code_voie             73785 non-null  object 
 9   code_postal                   73785 non-null  float64
 10  code_commune                  74593 non-null  int64  
 11  nom_commune                   74593 non-null  object 
 12  code_departement              74593 non-null  int64  
 13  a

Nettoyage général

In [17]:
# Standardiser les noms de colonnes
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Supprimer les colonnes entièrement vides
df = df.dropna(axis=1, how="all")

# Supprimer les doublons
df = df.drop_duplicates()

# Supprimer les colonnes trop vides (> 75% de valeurs manquantes)
missing_ratio = df.isnull().mean()
df = df.loc[:, missing_ratio < 0.75]

# Convertir en minuscules pour harmoniser les textes
for col in df.select_dtypes(include="object"):
    df[col] = df[col].str.lower()

Conversion des types et traitement des codes

In [18]:
# Conversion des dates
for col in df.columns:
    if "date" in col:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Code commune au format texte (5 chiffres)
if "code_commune" in df.columns:
    df["code_commune"] = df["code_commune"].astype(str).str.zfill(5)

Nettoyage des valeurs aberrantes


In [19]:
# Colonnes numériques principales
num_cols = ["valeur_fonciere", "nombre_lots", "surface_reelle_bati",
            "nombre_pieces_principales", "surface_terrain"]

for col in num_cols:
    if col in df.columns:
        # Remplacer valeurs négatives par 0
        df[col] = df[col].apply(lambda x: max(x, 0) if pd.notnull(x) else x)
        # Remplacer les NaN par la médiane
        df[col] = df[col].fillna(df[col].median())

Agrégation par commune

In [20]:
agg_dvf = (
    df.groupby("code_commune", as_index=False)
    .agg(
        nb_ventes=("valeur_fonciere", "count"),
        prix_median=("valeur_fonciere", "median"),
        prix_moyen=("valeur_fonciere", "mean"),
        surface_moyenne=("surface_reelle_bati", "mean"),
        pieces_moyennes=("nombre_pieces_principales", "mean")
    )
)

# Arrondir pour la lisibilité
agg_dvf[["prix_median", "prix_moyen", "surface_moyenne", "pieces_moyennes"]] = \
    agg_dvf[["prix_median", "prix_moyen", "surface_moyenne", "pieces_moyennes"]].round(0)

print(f"✅ Agrégation terminée : {agg_dvf.shape[0]} communes trouvées")

# Aperçu
display(agg_dvf.head(10))

✅ Agrégation terminée : 276 communes trouvées


,code_commune,nb_ventes,prix_median,prix_moyen,surface_moyenne,pieces_moyennes
0,69001,25,280000.0,223615.0,107.0,3.0
1,69002,24,94000.0,134785.0,75.0,1.0
2,69003,119,231000.0,311187.0,79.0,1.0
3,69004,24,244699.0,260873.0,86.0,2.0
4,69005,20,380000.0,301314.0,201.0,2.0
5,69006,258,133000.0,235563.0,80.0,1.0
6,69007,104,200000.0,317162.0,82.0,0.0
7,69008,12,39500.0,114767.0,74.0,1.0
8,69009,271,231000.0,272628.0,86.0,1.0
9,69010,193,227950.0,235016.0,76.0,1.0


Export 


In [21]:
agg_dvf.to_csv("../csv_clean/valeurs_foncieres_2024_clean.csv", index=False)
# print("✅ Exporté : valeurs_foncieres_2024_clean.csv")